# Utiliser des modèles

De plus en plus de modèles préentrainés

## Charger les données

In [ ]:
import pandas as pd
df = pd.read_csv("../data/dataframe.csv")
df.head()

## Une approche intégrée avec SpaCy

- `SpaCy` a des modèles entraînés pour les NER
- Par exemple pour le français, [plusieurs modèles sont disponibles](https://spacy.io/models/fr)
    - Avec des architectures différentes
- Une bibliothèque qui donne un framework commun.

::: {.callout-note appearance="simple" title="NER"}

Une étape : télécharger des modèles

https://github.com/explosion/spacy-models/releases/tag/fr_core_news_md-3.8.0

In [ ]:
#!python -m spacy download fr_core_news_md

Les utiliser

In [ ]:
df.loc[0, "texte"]

In [ ]:
import spacy
from spacy import displacy

nlp = spacy.load("fr_core_news_md")
doc = nlp(df.loc[0, "texte"])
displacy.render(doc, style="ent", jupyter=True)

### Manipuler les représentations du texte

In [ ]:
for token in doc[0:10]:
    print(f"{token.text:<15} | lemma: {token.lemma_} | POS: {token.pos_}")

In [ ]:
for ent in doc.ents[0:10]:
    print(f"{ent.text:<25} | label: {ent.label_}")

In [ ]:
displacy.render(doc[0:100], style="dep", jupyter=True, options={"compact": True})


### Un usage : récupérer uniquement les verbes

In [ ]:
def get_verbs(doc):
    """
    Get the verbs from a spacy doc
    """
    return [token.lemma_ for token in doc if token.pos_ == "VERB"]

tmp =  df["texte"][0:100].apply(lambda x: get_verbs(nlp(x)))

In [ ]:
from collections import Counter

Counter([j for i in tmp for j in i if j]).most_common(20)

### Un autre usage : nettoyer des textes

Plutôt que faire de la tokenisation brutale, nettoyer en prenant uniquement les lemmes puis faire un TF-IDF

In [ ]:
# exercice

## Aller utiliser d'autres modèles sur huggingface

Commençons par faire un tour sur Huggingface

### Prédire des entités nommées

Utilsons le modèle [GliNER disponible sur HuggingFace](https://github.com/urchade/GLiNER)

Ou sa version plus récente [GliNer](https://huggingface.co/knowledgator/gliner-multitask-large-v0.5)

In [ ]:
from gliner import GLiNER

model = GLiNER.from_pretrained("knowledgator/gliner-multitask-large-v0.5")

text = df.loc[0, "texte"]

labels = ["politicien ou politicienne"]

entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

### De nombreux modèles et la possibilité d'en entraîner

PAr exemple : https://huggingface.co/NousResearch/Minos-v1

### Analyse de sentiment

Une question : **quelles sont les prises de paroles les plus négatives ?**

- Embarras du choix
    - Par ex : [🚀 distilbert-based Multilingual Sentiment Classification Model
](https://huggingface.co/tabularisai/multilingual-sentiment-analysis)
- Comprendre le modèle / ce qu'il fait
- Importance d'évaluer son résultat

## Utiliser des modèles extérieurs

Il faut un endpoint :

- Ollama
- OpenAI
- ...

In [ ]:
import requests
import json

# Définir la prompt de classification
prompt = """Est-ce que ce texte est positif ou négatif : Je ne comprends rien au NLP"""

# Envoi de la requête à Ollama
response = requests.post(
    'https://llm-tools-alpha.huma-num.fr/ES_hpEC1VnleR_8C52IbSxHF/api/generate',
    json={
        'model': 'llama3.3',
        'prompt': prompt,
        'stream': False  # stream=False pour avoir une réponse simple
    }
)

# Traitement de la réponse
result = response.json()
result